# Project Report #
## About Dataset ##
I decided to choose the [Titanic data](https://www.udacity.com/api/nodes/5420148578/supplemental_media/titanic-datacsv/download) just out of curiosity.
Data contains the demographics and passenger information from 891 of the 2224 passengers and crew on board the Titanic. Description of the data can be found [here](https://www.kaggle.com/c/titanic/data)

## Questions  ##
* How the survival of the titanic boat passenger is related to his sex, age? In general we can assume that Females were given priority over Males in the rescue process.   
* Do other factors like **sibsp**(Number of Siblings/Spouses Aboard), **parch**(Number of Parents/Children Aboard), **pcalss**(Passenger Class) affect the survival rate ?

## Data Wrangling  ##


In [33]:
import pandas

data_file = 'titanic_data.csv'
# reading data file
titanic = pandas.read_csv(data_file)
# Top 5 rows of titanic
print titanic.head(5)
# High level descriptors of the data
titanic.describe()

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex  Age  SibSp  \
0                            Braund, Mr. Owen Harris    male   22      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female   38      1   
2                             Heikkinen, Miss. Laina  female   26      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   35      1   
4                           Allen, Mr. William Henry    male   35      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Missing Data ###
The *describe(titanic)* gives some important information about the data like count, mean, min, max, standard deviation of every column.
Looking into the **Age** columns in output above, we can see that count is *`714`* and for others it is *891*. So there are missing cells in Age. As **Age** can be an important factor influencing the survival, so we can't ignore it. We should fill some value in all the empty cells of **Age** column. Here , we will put median in emply cells. Pandas fillna function can be used in this scenario. 


In [34]:
# Fill empty cells of Age column with median
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200
